In [3]:
import pickle
import pandas as pd
import numpy as np

In [41]:
def isoform_in_window(starts, w_s, w_e):
    #print(starts, w_s, w_e)
    return any([x >= w_s and x <= w_e for x in starts])

In [4]:
np.random.seed(73298)

In [24]:
generated_dataset = pickle.load(open("second_dataset.pickle", "rb"))

In [25]:
generated_dataset = [x for x in generated_dataset if max(x['density']) > 10]

In [26]:
generated_dataset[:100]

[{'chrom': 'chrI',
  'strand': '+',
  'gene': 'YAL066W',
  'isoforms': [<simulate_reads.Isoform at 0x7f11bdfe2470>,
  'distr': array([0.25600663, 0.13504557, 0.6089478 ]),
  'density': [61,
   125,
   183,
   247,
   301,
   361,
   421,
   485,
   549,
   609,
   670,
   727,
   778,
   841,
   895,
   960,
   1020,
   1079,
   1141,
   1211,
   1266,
   1343,
   1391,
   1443,
   1500,
   1563,
   1628,
   1685,
   1747,
   1815,
   1882,
   1937,
   2013,
   2056,
   2122,
   2195,
   2250,
   2318,
   2381,
   2449,
   2504,
   2578,
   2641,
   2706,
   2768,
   2766,
   2769,
   2770,
   2783,
   2810,
   2819,
   2831,
   2819,
   2823,
   2826,
   2830,
   2832,
   2854,
   2854,
   2868,
   2886,
   2887,
   2909,
   2915,
   2927,
   2961,
   2961,
   2985,
   3011,
   3019,
   3022,
   3023,
   3046,
   3066,
   3086,
   3095,
   3101,
   3108,
   3148,
   3163,
   3160,
   3178,
   3188,
   3208,
   3218,
   3255,
   3281,
   3294,
   3300,
   3297,
   3315,
   3322,
   333

In [8]:
import helper
genome = helper.organize_genome_by_chrom(helper.read_bedfile())

INFO:helper:Reading bedfile...
INFO:helper:Done reading bedfile ../indexes/yeast-all.bed


In [55]:
window = 50
colnames = [str(i) for i in range(-window, window+1)] + ['Isoform']

labeled_data = []

keep_non_isoforms_threshold = 1

In [56]:
for entry in generated_dataset: 
    isoform_starts = [x.start for x in entry['isoforms']]
    isoform_starts = sorted(isoform_starts)
    gene_start = isoform_starts[0]
    utr_end = int(genome[entry['chrom']][entry['gene']].fields[6])
    utr_end = min(utr_end, isoform_starts[-1])
    for pos in range(gene_start + window, utr_end - window + 1):
        ws = (pos-window) - gene_start # density array is indexed from 0. pos 0 is gene start
        we = (pos+window+1) - gene_start 
        
        isoform =  int(isoform_in_window(isoform_starts, pos-window, pos+window+1))
        if isoform == 0:
            if np.random.uniform() > keep_non_isoforms_threshold:
                continue
        row = entry['density'][ws:we] + [isoform]
    
        labeled_data.append(dict(zip(colnames, row)))

In [58]:
labeled_data = pd.DataFrame(labeled_data)
labeled_data['-50'].describe()

count    159872.000000
mean         18.472528
std          93.493307
min           0.000000
25%           1.000000
50%           4.000000
75%          11.000000
max        3927.000000
Name: -50, dtype: float64

In [59]:
labeled_data['Isoform'].value_counts()

0    82050
1    77822
Name: Isoform, dtype: int64

In [60]:
from sklearn.model_selection import train_test_split

In [61]:
y = labeled_data.pop('Isoform')
X = labeled_data

In [62]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=1)

In [63]:
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

In [67]:
classifier = Sequential()
classifier.add(Dense(units = 101, activation='sigmoid', input_dim=101))
classifier.add(Dense(units = 202, activation='relu'))
classifier.add(Dense(units = 102, activation='relu'))
classifier.add(Dense(units = 1, activation='sigmoid'))
classifier.compile(optimizer='sgd', loss='binary_crossentropy')



In [69]:
classifier.fit(X_train, y_train, batch_size=30, epochs=30)

Epoch 1/30
95923/95923 [==============================] - 8s 83us/step - loss: 0.5676
Epoch 2/30
95923/95923 [==============================] - 8s 85us/step - loss: 0.5551
Epoch 3/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5567
Epoch 4/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5548
Epoch 5/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5536
Epoch 6/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5540
Epoch 7/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5531
Epoch 8/30
95923/95923 [==============================] - 7s 77us/step - loss: 0.5552
Epoch 9/30
95923/95923 [==============================] - 6s 67us/step - loss: 0.5590
Epoch 10/30
95923/95923 [==============================] - 7s 76us/step - loss: 0.5554
Epoch 11/30
95923/95923 [==============================] - 7s 78us/step - loss: 0.5577
Epoch 12/30
95923/95923 [===========================

In [74]:
y_pred = classifier.predict(X_test)
y_pred = [1 if Y >= 0.5 else 0 for Y in y_pred]


In [71]:
for i in range(0, len(y_pred)):
    if int(y_pred[i]) == 1 and int(y_test.iloc[i]) == 1:
        print(X_test.iloc[i])

-50     1
-49     0
-48     0
-47     0
-46     0
       ..
46     19
47     19
48     19
49     19
50     18
Name: 130360, Length: 101, dtype: int64
-50     1
-49     2
-48     2
-47     2
-46     2
       ..
46     12
47     13
48     15
49     15
50     15
Name: 146745, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     10
47     10
48     12
49     12
50     12
Name: 111694, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     3
-46     2
       ..
46     29
47     29
48     29
49     28
50     28
Name: 52063, Length: 101, dtype: int64
-50    13
-49    13
-48    13
-47    13
-46    12
       ..
46     80
47     79
48     77
49     76
50     75
Name: 90827, Length: 101, dtype: int64
-50     4
-49     4
-48     5
-47     5
-46     5
       ..
46     60
47     61
48     63
49     66
50     65
Name: 107120, Length: 101, dtype: int64
-50    15
-49    15
-48    14
-47    16
-46    16
       ..
46     30
47     31
48     31
49     31
50

Name: 107157, Length: 101, dtype: int64
-50     24
-49     24
-48     24
-47     26
-46     26
      ... 
46     169
47     171
48     170
49     167
50     164
Name: 141985, Length: 101, dtype: int64
-50     1
-49     1
-48     2
-47     2
-46     2
       ..
46     41
47     41
48     41
49     43
50     43
Name: 126079, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     3
-46     2
       ..
46     18
47     18
48     19
49     19
50     19
Name: 33883, Length: 101, dtype: int64
-50    14
-49    14
-48    14
-47    13
-46    13
       ..
46     61
47     61
48     63
49     64
50     61
Name: 21271, Length: 101, dtype: int64
-50     2
-49     2
-48     2
-47     2
-46     2
       ..
46     14
47     14
48     15
49     16
50     16
Name: 94053, Length: 101, dtype: int64
-50    17
-49    17
-48    16
-47    16
-46    12
       ..
46     33
47     37
48     41
49     42
50     43
Name: 400, Length: 101, dtype: int64
-50     228
-49     228
-48     228
-47     227
-46    

Name: 5965, Length: 101, dtype: int64
-50    25
-49    24
-48    27
-47    29
-46    30
       ..
46     92
47     90
48     88
49     89
50     91
Name: 106378, Length: 101, dtype: int64
-50      7
-49      8
-48     11
-47     14
-46     18
      ... 
46     217
47     219
48     219
49     217
50     218
Name: 147029, Length: 101, dtype: int64
-50    12
-49    11
-48    10
-47    10
-46    10
       ..
46     21
47     21
48     21
49     19
50     22
Name: 28233, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     39
47     39
48     39
49     37
50     39
Name: 94084, Length: 101, dtype: int64
-50     9
-49     9
-48     9
-47     9
-46     9
       ..
46     22
47     23
48     23
49     23
50     23
Name: 152911, Length: 101, dtype: int64
-50    13
-49    13
-48    14
-47    14
-46    14
       ..
46     39
47     39
48     39
49     41
50     42
Name: 71830, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
    

Name: 59211, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     12
47     12
48     12
49     13
50     14
Name: 88493, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     3
-46     3
       ..
46     11
47     11
48     11
49     12
50     12
Name: 139899, Length: 101, dtype: int64
-50     23
-49     24
-48     26
-47     28
-46     31
      ... 
46     287
47     283
48     284
49     288
50     289
Name: 115171, Length: 101, dtype: int64
-50    0
-49    0
-48    0
-47    0
-46    0
      ..
46     4
47     4
48     4
49     4
50     4
Name: 17569, Length: 101, dtype: int64
-50     1
-49     0
-48     0
-47     0
-46     1
       ..
46     32
47     33
48     33
49     34
50     32
Name: 147736, Length: 101, dtype: int64
-50     120
-49     141
-48     166
-47     184
-46     208
       ... 
46     2122
47     2140
48     2161
49     2166
50     2183
Name: 107571, Length: 101, dtype: int64
-50     4
-49     4
-48     4
-47     4
-

Name: 49955, Length: 101, dtype: int64
-50    1
-49    1
-48    1
-47    1
-46    1
      ..
46     8
47     8
48     9
49     9
50     9
Name: 73259, Length: 101, dtype: int64
-50     2
-49     3
-48     4
-47     4
-46     4
       ..
46     42
47     43
48     44
49     44
50     45
Name: 158357, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
       ..
46     15
47     15
48     15
49     15
50     15
Name: 132233, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
       ..
46     11
47     11
48     12
49     12
50     14
Name: 98750, Length: 101, dtype: int64
-50    0
-49    0
-48    0
-47    0
-46    0
      ..
46     6
47     6
48     6
49     6
50     6
Name: 120448, Length: 101, dtype: int64
-50     7
-49     8
-48     8
-47     9
-46     9
       ..
46     30
47     30
48     31
49     32
50     32
Name: 76353, Length: 101, dtype: int64
-50    0
-49    0
-48    0
-47    0
-46    0
      ..
46     3
47     3
48     4
49   

...

Name: 157636, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     4
-46     4
       ..
46     29
47     31
48     30
49     29
50     28
Name: 11537, Length: 101, dtype: int64
-50    0
-49    0
-48    0
-47    0
-46    0
      ..
46     7
47     7
48     7
49     7
50     6
Name: 86737, Length: 101, dtype: int64
-50      8
-49      8
-48      8
-47     10
-46     11
      ... 
46     118
47     118
48     117
49     118
50     116
Name: 93355, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
       ..
46     10
47      9
48      9
49      8
50      8
Name: 92401, Length: 101, dtype: int64
-50    197
-49    204
-48    208
-47    212
-46    216
      ... 
46     510
47     519
48     521
49     524
50     538
Name: 136227, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
       ..
46     16
47     16
48     16
49     16
50     16
Name: 85147, Length: 101, dtype: int64
-50    12
-49    13
-48    14
-47    14
-46    13
    

Name: 96090, Length: 101, dtype: int64
-50    0
-49    0
-48    0
-47    0
-46    0
      ..
46     4
47     4
48     4
49     4
50     4
Name: 97147, Length: 101, dtype: int64
-50     0
-49     0
-48     0
-47     0
-46     0
       ..
46     17
47     18
48     18
49     18
50     19
Name: 71141, Length: 101, dtype: int64
-50     51
-49     50
-48     50
-47     53
-46     53
      ... 
46     113
47     114
48     115
49     120
50     120
Name: 52549, Length: 101, dtype: int64
-50    20
-49    21
-48    22
-47    22
-46    23
       ..
46     51
47     55
48     61
49     61
50     60
Name: 134791, Length: 101, dtype: int64
-50    39
-49    40
-48    41
-47    40
-46    39
       ..
46     80
47     81
48     81
49     84
50     82
Name: 53339, Length: 101, dtype: int64
-50    15
-49    17
-48    17
-47    17
-46    17
       ..
46     42
47     42
48     42
49     41
50     42
Name: 89805, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46    

Name: 144133, Length: 101, dtype: int64
-50     1
-49     2
-48     2
-47     2
-46     2
       ..
46     31
47     32
48     33
49     35
50     35
Name: 158348, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     3
-46     3
       ..
46     31
47     31
48     32
49     33
50     33
Name: 24525, Length: 101, dtype: int64
-50     3
-49     3
-48     3
-47     3
-46     3
       ..
46     11
47     11
48     11
49     11
50     11
Name: 48695, Length: 101, dtype: int64
-50    16
-49    17
-48    16
-47    16
-46    16
       ..
46     43
47     42
48     43
49     43
50     44
Name: 137212, Length: 101, dtype: int64
-50    1054
-49    1107
-48    1154
-47    1207
-46    1265
       ... 
46     6085
47     6090
48     6119
49     6147
50     6154
Name: 5992, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     10
47     10
48     10
49     10
50     10
Name: 139356, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     2
-4

Name: 89250, Length: 101, dtype: int64
-50    15
-49    17
-48    16
-47    17
-46    17
       ..
46     38
47     36
48     39
49     40
50     41
Name: 121477, Length: 101, dtype: int64
-50     2
-49     2
-48     2
-47     2
-46     2
       ..
46     16
47     15
48     15
49     15
50     16
Name: 75311, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     11
47     11
48     10
49     10
50      9
Name: 75031, Length: 101, dtype: int64
-50      9
-49      9
-48      9
-47     10
-46     10
      ... 
46     168
47     168
48     169
49     173
50     176
Name: 150963, Length: 101, dtype: int64
-50     4
-49     4
-48     4
-47     4
-46     4
       ..
46     12
47     14
48     14
49     14
50     14
Name: 24102, Length: 101, dtype: int64
-50     1
-49     1
-48     1
-47     1
-46     1
       ..
46     11
47     11
48     11
49     11
50     11
Name: 120771, Length: 101, dtype: int64
-50    1
-49    1
-48    1
-47    1
-46    1
      ..

KeyboardInterrupt: 

In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.7215750050821749

In [76]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, y_pred)

array([[25331,  7541],
       [10264, 20813]])